# Common OSINT Model Examples
This notebook shows how to use the common model. In order to do this "live", you need access to BinaryEdge, Censys
and/or Shodan. You can also use data in the [test_data](test_data) directory.

**Note:** The modules used in this notebook are not installed by default when installing common-osint-model as they are
no direct project requirements. In order to run all the examples code, you need to (pip) install:
  - shodan
  - pybinaryedge
  - censys

# Initialize API clients
The following cells set-up the api clients in order to grab live data from BinaryEdge, Censys and Shodan. **This is
currently not implemented and work in progress. Please skip the following cell for now.**

In [ ]:
shodan_key = input("Shodan API-Key (empty to skip downloading data from Shodan)")
be_key = input("BinaryEdge API-Key")
censys_uid = input("Censys UID")
censys_secret = input("Censys Secret")
clients = {}
if shodan_key and len(shodan_key) > 0:
    import shodan
    clients["shodan"] = shodan.Shodan(shodan_key)

if be_key and len(be_key) > 0:
    import pybinaryedge
    clients["binaryedge"] = pybinaryedge.BinaryEdge(be_key)

if censys_uid and censys_secret and len(censys_uid) > 0 and len(censys_secret) > 0:
    from censys.search.v2 import CensysHosts
    clients["censys"] = CensysHosts(censys_uid, censys_secret)

print(f"Initialized {len(clients)} API(s).")

# Play around with test data

In [1]:
import json
from common_osint_model import Host
binaryedge_host = Host.from_binaryedge(json.loads(open("test_data/140.82.121.4_binaryedge.json").read()))
shodan_host = Host.from_shodan(json.loads(open("test_data/140.82.121.4_shodan.json").read()))
censys_host = Host.from_censys(json.loads(open("test_data/140.82.121.4_censys_v2.json").read()))

print("=== FEATURES ===\n")
print("Access attributes directly, no need to use dictionaries:")
print(f"IP: {binaryedge_host.ip}")
print(f"Service on port {binaryedge_host.services[0].port}: {binaryedge_host.services[0].banner}\n")

print(f"Automatic calculation of additional hashes:")
print(f"Other content hashes for Shodan: {shodan_host.services[0].http.content.sha256}")
print(f"Other SSH fingerprints for Censys data (e.g. Murmur): {censys_host.services[0].ssh.key.murmur}\n")

print("Extracting domains returned by the data sources as their own objects. This way they can be extended with, e.g., "
      "passive dns data")
print(f"Extracted Domains (Shodan): ")
for d in shodan_host.domains:
    print(f"  - Shodan: {d.domain} (type: {d.type})")
print("\nExtracted Domains (Censys): ")
for d in censys_host.domains:
    print(f"  - Censys: {d.domain} (type: {d.type})")

print("\nEasy export of data into json:")
print(shodan_host.flattened_json())

INFO:SSHComponentKey:Censys data does not contain the key as raw data. The public key can be constructed with given data, however, currently this is only supported for RSA keys.
INFO:TLSComponentCertificate:Censys does not provide raw certificate data, to hashes must be taken from the data and cannot be calculated.


=== FEATURES ===

Access attributes directly, no need to use dictionaries:
IP: 140.82.121.4
Service on port 443: HTTP/1.1 301 Moved Permanently\r\nContent-Length: 0\r\nLocation: https://github.com/\r\nconnection: close\r\n\r\n

Automatic calculation of additional hashes:
Other content hashes for Shodan: e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
Other SSH fingerprints for Censys data (e.g. Murmur): -388505952

Extracting domains returned by the data sources as their own objects. This way they can be extended with, e.g., passive dns data
Extracted Domains (Shodan): 
  - Shodan: github.com (type: domain)
  - Shodan: lb-140-82-121-4-fra.github.com (type: rdns)
  - Shodan: www.github.com (type: common_name)

Extracted Domains (Censys): 
  - Censys: www.github.com (type: common_name)
  - Censys: github.com (type: common_name)

Easy export of data into json:
{
  "443.port": 443,
  "443.banner": "HTTP/1.1 301 Moved Permanently\r\nContent-Length: 0\r\nLocation: https://gi